<strong>[1] Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those? [relevant rubric items: “data exploration”, “outlier investigation”]</strong>



The Enron scandal is the single biggest financial fraud in the history, which resulted in the shares once worth \$90.75 in December 2001 dropping to \$0.67 in mere 2 months. Numerous executives and accountants were prosecuted, while many more suffered from the collapse, losing jobs and large sums of money.

The goal of this project is to identify the Persons Of Interest (POIs), referring to those convicted, indicted, or have testified in the court, based on publically available financial and email data, using machine learning.

The original dataset available has 146 observations, and of them 18 are labeled as POIs. There are 20 predictor variables, and 1 target variable, which the POI label.
There were three outliers that I got rid of during the exploration stage. One is ‘THE TRAVEL AGENCY IN THE PARK’, which is the travel agency for the company, not an employee. The second is 'LOCKHART EUGENE E', who only had NaN values for predictive variables, and thus was not helpful for the analysis. The last one was ‘TOTAL’, which is a sum of all the values in the dataset. Removing the ‘TOTAL’ also revealed that the true correlations among variables, which seemed all highly correlated with the outlier. 

An interesting founding from the data exploration stage was that there was a lot of missing values for many variables, which would probably have an effect on the machine learning process and outcome.  

<strong>[2] What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values. [relevant rubric items: “create new features”, “intelligently select features”, “properly scale features”]</strong>

In the end, I used the following 9 features for my POI identifier (in order of importance) :

'exercised_stock_options' = 24.82

'total_stock_value' = 24.18

'bonus' = 20.79

'salary' = 18.29

'deferred_income' = 11.46

'long_term_incentive' = 9.92

'restricted_stock' = 9.21

'stock_salary_proportion' = 9.18

'total_payments' = 8.77

I have used the SelectKBest module from sklearn to evaluate 7, 9, 10, 11, 12, 13 features for each algorithm, and found that 9 variables produced not only the highest f1 score with GaussanNB model, but also was one of the few that even managed to produce both precision and recall scores over 0.3. I also scaled the data using the MinMaxScaler(). Financial variables like ‘salary’ had extremely large values when especially compared to email variables such as ‘shared_receipt with_poi’, so scaling was crucial in order not to give too much weight to financial values for the actual machine learning stage.

 I created one new variable called ‘stock_salary_proportion’ for this dataset. My hypothesis was that people with higher proportion of stock to salary (hard cash) will have more interest in increasing the arbitrary value of the company, because they can then benefit from the equally increased stock value. I created this value by dividing the stock total stock value by the sum of stock and salary values. In the end, this feature was part of the 9 best features used to create the identifier, though I suspect one of the reason may be that the two variables used to create this new feature were also highly important to build the final algorithm.

<strong>[3] What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms? [relevant rubric item: “pick an algorithm”]</strong>

I have tested all the supervised learning classifiers, which are 'Decision Tree Classifier', 'Random Forest Classifier', 'AdaBoost Classifier', 'Support Vector Machine Classifier', 'Gaussian Naive Bayes', and 'K Nearest Neighbors'. The initial test runs of the algorithms without any parameter tuning showed that Gaussian Naïve Bayes had the best results in all accuracy, precision, and recall scores. However, because I haven’t used any parameters and only used a single fold train_test_split, I wanted to test all the algorithms again with certain parameter tuning. 

With the parameters, the ensemble classifiers and the SVC were incredibly slow. Overall, knn was the worst performing model even with the tuning, and I suspect that this is mainly because some POIs (like Kenneth Lay) had extreme values even within the group of POIs, and thus the 'neighboring' observations were not really useful in defining all the POIs.

In the end, I chose GaussianNB with 9 features. Despite no tuning, GaussianNB showed impressive results, and was actually the only algorithm with both recall and precision scores above 0.3 (recall 0.32, precision 0.35).

<strong>[4] What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well? How did you tune the parameters of your particular algorithm? What parameters did you tune? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier). [relevant rubric items: “discuss parameter tuning”, “tune the algorithm”]
</strong>

In the sense of machine learning, tuning the parameters of an algorithm means optimizing the algorithm for better performance. From what I see it, there are two main objectives for ‘optimization’. The first is higher precision/recall score, and the second is faster processing time. For this project, I focused on optimizing for higher precision/recall score, because it is a relatively small dataset, and processing time does not really have an effect on the goal of this project, which is to correctly identify POIs. 

I have used the GridSearchCV module to find optimal parameters for multiple algorithms. These are the parameters I tried to tune for each algorithm:

- Decision Tree: criterion, max_depth, max_features, min_samples_leaf, min_samples_split
- Random Forest: criterion, max_features, min_samples_leaf, n_estimators
- AdaBoost: algorithm, learning_rate, n_estimators
- SVC(kernel = 'rbf' fixed because there were only a few variables used for this algorithm): C, gamma
- KNN: algorithm, metric, n_neighbors, weights

In the end, because I chose Gaussian Naive Bayes as my classifier, I haven't used any parameter tuning for the final identifier. However, I did try to tweak the model by testing it with different number of features, and ended up using 9 features, which showed the best performance using GaussianNB.

<strong>[5] What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis? [relevant rubric items: “discuss validation”, “validation strategy”]
</strong>

Validation is to test whether the result we have seen in our training stage can be generalized in other, new datasets.

A classic mistake often made at this stage is not to separate training and test data. This doesn't allow us to really validate whether the model will show similar results if new datasets were to be used in the future. However, another mistake that can be made with a really small dataset like this is only splitting the dataset once. I have actually made this mistake by only dividing the dataset once using the train_test_split() module. Because there were only 18 POIs, in worst case scenario, only 1 or 2 POIs might end up in the test set. Thus, cross-validation methods such as StratifiedShuffleSplit with sufficient folds should be used for such smaller datasets.

I used the StratifiedShuffleSplit with 10 splits and 10% of data allocated to test data to validate the tuned algorithms.


<strong>[6] Give at least 2 evaluation metrics and your average performance for each of them. Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]
</strong>

I have constantly used 3 different metrics; the recall score, precision score, and the f1 score. For the Enron case, the recall score indicates the ability to detect POI among the group, and the precision score shows what proportion of the people tagged as POI by the classifier is truly a POI. f1 is just an average of the two scores.

I initially wanted a model that has a higher recall score in the expense of a loewr precision score, because it is better safe than sorry and let go of potential POIs. In the end, all of the models I have tested had higher precision scores, so I had to settle with the GaussianNB model which was the only model with both scores over 0.3 (precision 0.38, recall 0.32). This means that the model's prediction is slightly more reliable, but it may have let go of possible POIs for making safe bets.
